<center>
<img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/assets/logos/SN_web_lightmode_vertical.png" width="300" alt="cognitiveclass.ai logo"  />
</center>

# Lab 6. Data Analysis with Python

Estimated time needed: **30** minutes

## Objectives

After completing this lab you will be able to:

*   Be confident about your data analysis skills

The data that we are going to use for this is a subset of an open source diabetes in US DataSet: https://www.kaggle.com/datasets/akshaydattatraykhare/diabetes-dataset.

This dataset is from the National Institute of Diabetes and Digestive and Kidney Diseases. The objective of the dataset is to diagnostically predict whether a patient has diabetes based on certain diagnostic measurements included in the dataset. You will be asked to analyze the data and predict the diagnosis. At the end of the lab, you will be instructed on how you can share your notebook.


## Table of Contents

<div class="alert alert-block alert-info" style="margin-top: 20px">
    <ul>
        <li>Import libraries</li>
        <li>Importing the Data</li>
        <li>Question 1: Display the data types of each column using the attribute `dtypes`.</li>
        <li>Question 2: change datatypes to the correct one</li>
        <li>Question 3: build correlation heatmap</li>
        <li>Question 4: separate dataset into two dataframes for the feature column and the target column</li>
        <li>Question 5: separate DataSets for train and test DataSets for calculate accuracy of models.</li>
        <li>Question 6: create transformer</li>
        <li>Question 7: create logistic regression pipline</li>
        <li>Question 8: fit logistic regression pipline</li>
        <li>Question 9: get scores for train and test DataSets from pipline</li>
        <li>Question 10: add cross-validation</li>
        <li>Question 11: create ensemble of classifiers using</li>
        <li>Question 12: show the train and test scores of an ensemble</li>
        <li>Question 13: build decision tree</li>
        <li>Question 14: show text representation of a built decision tree</li>
        <li>Question 15: plot decision tree</li>
        <li>Question 16: create a function that will predict patients diagnoses based on input data</li>
    </ul>
</div>

You will need the following libraries:

In [ ]:
!pip install scikit-learn
!pip install imblearn
!pip install graphviz

## Import libraries

In [ ]:
import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
from sklearn.compose import make_column_transformer
from sklearn.preprocessing import StandardScaler
from sklearn import set_config

from imblearn.pipeline import make_pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import cross_val_predict
from sklearn.model_selection import train_test_split


from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import VotingClassifier
from sklearn.metrics import recall_score, precision_score
import time

from sklearn import tree
import graphviz


This function will download the dataset into your browser 


## Importing the Data

you will need to download the dataset; if you are running locally, please comment out the following 


In [ ]:
path = 'https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMSkillsNetwork-GPXX08R2EN/NewDiabetes.csv'

Load the csv:


In [ ]:
df= pd.read_csv(path)

We use the method  <code>head()</code>  to display the first 5 columns of the dataframe:


In [ ]:
df.head()

<details>
<summary><b>Click to see attribute information</b></summary>
Input features (column names):

1. `Pregnancies` - express the Number of pregnancies . (int64)
2. `Glucose` - express the Glucose level in blood. (int64)
3. `BloodPressure` - express the Blood pressure measurement. (int64)
4. `SkinThickness` - xpress the thickness of the skin. (int64)
5. `Insulin` - express the Insulin level in blood. (int64)
6. `BMI` - express the Body mass index. (float64)
7. `DiabetesPedigreeFunction` - express the Diabetes percentage. (float64)
8. `Age` - express the age. (int64)
9. `Outcome`  [Target Column]  - express the final result 1 is Yes and 0 is No. (boolean)

</details>

In [ ]:
df.info()

<b>Question 1</b>:  display the data types of each column using the attribute `dtypes`.

In [ ]:
df.dtypes

<b>Question 2 </b>:  change datatypes to the correct one.

In [ ]:
df['Outcome'] = df['Outcome'].astype('bool')
df.dtypes

<b>Question 3 </b>:  build correlation heatmap.

In [ ]:
corrmat = df.corr()
top_corr_features = corrmat.index
plt.figure(figsize=(8,7))
g=sns.heatmap(df.corr(),annot=True,cmap="RdYlGn")

<b>Question 4 </b>: separate dataset into two dataframes for the feature column and the target column.

In [ ]:
x = df.drop(columns = ['Outcome'])
y = df['Outcome']

<b>Question 5 </b>: separate DataSets for train and test DataSets for calculate accuracy of models.

In [ ]:
x_train,x_test,y_train,y_test = train_test_split(x,y,test_size = 0.33, shuffle=False)

<b>Question 6</b>: create transformer using **[make_column_transformer](https://scikit-learn.org/stable/modules/generated/sklearn.compose.make_column_transformer.html)**.

In [ ]:
trans = make_column_transformer((StandardScaler(),x.columns))
set_config(display = 'diagram')
trans

<b>Question 7</b>: create logistic regression pipline.

In [ ]:
lr = LogisticRegression()
pipe_lr = make_pipeline(trans,lr)


<b>Question 8</b>: fit logistic regression pipline.

In [ ]:
pipe_lr.fit(x_train,y_train)

<b>Question 9</b>: get scores for train and test DataSets from pipline.

In [ ]:
scores_train = pipe_lr.score(x_train, y_train)
scores_test = pipe_lr.score(x_test, y_test)
print('Training DataSet accuracy: {: .1%}'.format(scores_train), 'Test DataSet accuracy: {: .1%}'.format(scores_test))

<b>Question 10</b>: add cross-validation.

In [ ]:
Rcross = cross_val_score(pipe_lr,x ,y, cv = 4)
print(Rcross)
print("The mean of the folds are", Rcross.mean(), "and the standart deviation is", Rcross.std())

yhat = cross_val_predict(pipe_lr,x,y,cv = 4)
yhat[0:5]

<b>Question 11</b>: create ensemble of classifiers using.

In [ ]:
names = ["Logistic Regression",
         "Decision Tree", "Random Forest","Gaussian Naive Bayes"]
classifiers = [
    LogisticRegression(),
    DecisionTreeClassifier(max_depth=8),
    RandomForestClassifier(n_estimators=10, max_features=1),
    GaussianNB(),
]

scores_train = []
scores_test = []
scores_train_s = []
scores_test_s = []

In [ ]:
est = [(str(est), est) for est in classifiers]
eclf = [VotingClassifier(
    estimators=est,
    voting='hard')]
names += ["Voting Classifier"]
classifiers += eclf
for name, classif in zip(names, classifiers):
    start_time = time.time()
    print(name,'fitting.....',end = '')
    clf = make_pipeline(trans, classif)
    clf.fit(x_train,y_train)
    score_train = recall_score(y_train, clf.predict(x_train), average='micro')
    score_test = recall_score(y_test, clf.predict(x_test), average='micro')
    scores_train.append(score_train)
    scores_test.append(score_test)

    end_time = time.time()
    print(" [",round(end_time - start_time,2),"s]")

<b>Question 12</b>: show the train and test scores of an ensemble.

In [ ]:
res = pd.DataFrame(index = names)
res['Train'] = np.array(scores_train)
res['Test'] = np.array(scores_test)
res.index.name = "Classifier accuracy"
pd.options.display.float_format = '{:,.2f}'.format
res

In [ ]:
fig, ax = plt.subplots(figsize=(10,6))
ax.bar(names, scores_test)

ax.set_title('Calassifiers Accuracy')
ax.set_xlabel('Classifier')
ax.set_ylabel('Accuracy')

plt.xticks(rotation=45)
plt.show()

In [ ]:
clf

<b>Question 13</b>: build decision tree.

In [ ]:
model = DecisionTreeClassifier()

DTC = make_pipeline(trans, model)

DTC.fit(x_train, y_train)

scores_test = DTC.score(x_test, y_test)

# accuracy = accuracy_score(y_enc, yhat)
print('Accuracy: %.2f' % (scores_test * 100))

<b>Question 14</b>: show text representation of a built decision tree.

In [ ]:
text_representation = tree.export_text(model)
print(text_representation)

In [ ]:
with open("decistion_tree.log", "w") as fout:
    fout.write(text_representation)

<b>Question 15</b>: plot decision tree using **[tree.plot_tree](https://scikit-learn.org/stable/modules/generated/sklearn.tree.plot_tree.html)**

In [ ]:
fig = plt.figure(figsize=(100,100))
_ = tree.plot_tree(model,
                   max_depth = 4,
                   feature_names = x.columns,
                   class_names = y.unique().astype(str),
                   filled = True)

In [ ]:
dot_data = tree.export_graphviz(model,
                                max_depth = 7,
                                feature_names = x.columns,
                                class_names = y.unique().astype(str),
                                filled=True)

In [ ]:
graph = graphviz.Source(dot_data, format="png")


In [ ]:
graph.render("decision_tree_graphivz")

<b>Question 16</b>: create a function that will predict patients diagnoses based on input data.

In [ ]:
def fit(file):
    df2 = pd.read_csv(file)

    X = df2.drop(columns = ['Outcome'])
    Y = df2['Outcome']

    x_tr,x_te,y_tr,y_te = train_test_split(X,Y,test_size = 0.33, shuffle=False)
    transformer = make_column_transformer((StandardScaler(),x.columns))

    names = ["Logistic Regression",
             "Decision Tree", "Random Forest","Gaussian Naive Bayes"]
    
    classifiers = [
        LogisticRegression(),
        DecisionTreeClassifier(max_depth=8),
        RandomForestClassifier(n_estimators=10, max_features=1),
        GaussianNB(),
    ]
    
    est = list(zip(names, classifiers))
    
    Vote_clasif = VotingClassifier(
        estimators=est,
        voting='hard')

    pipel = make_pipeline(transformer, Vote_clasif)

    pipel.fit(x_tr, y_tr)

    score = pipel.score(x_te,y_te)
    print(f"Accuracy : {round(score,3)}")
    
    return pipel
    # score(y_train, clf.predict(x_train), average='micro')
    # return clf

In [ ]:
def pred(my_clf, input_df):

    predictions = pd.DataFrame(my_clf.predict(input_df))
    
    print(predictions[0].map({1: "Yes", 0: "No"}))


In [ ]:
# 6,148,72,35,0,33.6,0.627,50,1
# 1,85,66,29,0,26.6,0.351,31,0
# 8,183,64,0,0,23.3,0.672,32,1
# 1,89,66,23,94,28.1,0.167,21,0

input_df = pd.DataFrame(data=[[6,148,72,35,0,33.6,0.627,50],
                              [1,85,66,29,0,26.6,0.351,31],
                              [8,183,64,0,0,23.3,0.672,32],
                              [1,89,66,23,94,28.1,0.167,21]], columns=['Pregnancies', 'Glucose', 'BloodPressure', 'SkinThickness', 'Insulin', 'BMI', 'DiabetesPedigreeFunction', 'Age'])
my_clf = fit("https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMSkillsNetwork-GPXX08R2EN/NewDiabetes.csv")
pred(my_clf, input_df)

In [ ]:
clf

### Thank you for completing this lab!

## Author

<a href="https://author.skills.network/instructors/dmytro_yesyp" target="_blank">Dmytro Yesyp</a>

### Other Contributors

<a href="https://author.skills.network/instructors/yaroslav_vyklyuk_2">Prof. Yaroslav Vyklyuk, DrSc, PhD</a>

<a href="https://author.skills.network/instructors/nataliya_boyko">Ass. Prof. Nataliya Boyko, PhD</a>

## Change Log

| Date (YYYY-MM-DD) | Version | Changed By | Change Description                                         |
| ----------------- | ------- | ---------- | ---------------------------------------------------------- |
| 2023-04-01       | 01     | Dmytro Yesyp     | Lab created|
<hr>

## <h3 align="center"> © IBM Corporation 2023. All rights reserved. <h3/>

